In [1]:
##Branch and Bound!

import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import scipy.stats
from scipy.stats import norm
from sklearn.linear_model import LinearRegression  
import itertools
import copy


data = pd.read_csv("C:/Users/Praktikant1/Desktop/Projekte/Branch and Bound/Branch-and-Bound/dataset.csv",sep=";").astype(float)

In [2]:
#variables and speficicatons

headers = list(data)
Y = headers[0]
X = headers[1:]

In [3]:
regressor = LinearRegression()  

R = regressor.fit(data[X],data[Y])  
s = regressor.score(data[X], data[Y]) 

#prints mean squared error
print(s)

0.032779569202025516


In [ ]:
## exhaustive search? create class here

class bbalgorithm: 
    
    """
     def __init__(self, dependant, indepedant):
        self.depedant = dependant
        self.indepedant = independat
    
    """
    def combine():
        
    
    def branch():
        
        pass
    
    def bound():
        
        pass
    


In [8]:
## This function generates all possible combinations of the list!

def powerset(data):
    target = []
    
    def combinations(target,data):
        for i in range(len(data)):
            new_target = copy.copy(target)
            new_data = copy.copy(data)
            new_target.append(data[i])
            new_data = data[i+1:]
            pp.append(new_target)
            #print(new_target)
            combinations(new_target,new_data)
    pp = []
    rrr = [combinations(target, X)]
    return pp


In [9]:
result = powerset(X)

In [13]:
print(result)

[['SEX'], ['SEX', 'INCOME'], ['SEX', 'INCOME', 'EDUCATION'], ['SEX', 'INCOME', 'EDUCATION', 'AGE'], ['SEX', 'INCOME', 'EDUCATION', 'AGE', 'LOCATION'], ['SEX', 'INCOME', 'EDUCATION', 'LOCATION'], ['SEX', 'INCOME', 'AGE'], ['SEX', 'INCOME', 'AGE', 'LOCATION'], ['SEX', 'INCOME', 'LOCATION'], ['SEX', 'EDUCATION'], ['SEX', 'EDUCATION', 'AGE'], ['SEX', 'EDUCATION', 'AGE', 'LOCATION'], ['SEX', 'EDUCATION', 'LOCATION'], ['SEX', 'AGE'], ['SEX', 'AGE', 'LOCATION'], ['SEX', 'LOCATION'], ['INCOME'], ['INCOME', 'EDUCATION'], ['INCOME', 'EDUCATION', 'AGE'], ['INCOME', 'EDUCATION', 'AGE', 'LOCATION'], ['INCOME', 'EDUCATION', 'LOCATION'], ['INCOME', 'AGE'], ['INCOME', 'AGE', 'LOCATION'], ['INCOME', 'LOCATION'], ['EDUCATION'], ['EDUCATION', 'AGE'], ['EDUCATION', 'AGE', 'LOCATION'], ['EDUCATION', 'LOCATION'], ['AGE'], ['AGE', 'LOCATION'], ['LOCATION']]
